In [ ]:
import pandas as pd
import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf

from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

days = 1

stockTicker = 'TSLA'

In [ ]:
merged_df = pd.read_csv('./data/stock/' + stockTicker + '.csv')

In [ ]:
merged_df['future'] = merged_df['close'].shift(-days)
last_sequence = np.array(merged_df[['close']].tail(7))
merged_df.dropna(inplace=True)
sequence_data = []
sequences = deque(maxlen=N_STEPS)

for entry, target in zip(merged_df[['close'] + ['date']].values, merged_df['future'].values):
      sequences.append(entry)
      if len(sequences) == N_STEPS:
          sequence_data.append([np.array(sequences), target])
        
print("sequences1", last_sequence)


# construct the X's and Y's
X, Y = [], []
for seq, target in sequence_data:
    X.append(seq)
    Y.append(target)
# convert to numpy arrays

X = np.array(X)
X = np.delete(X,[1],2).astype(np.float32)
Y = np.array(Y).astype(np.float32)

In [ ]:
model = Sequential()
model.add(LSTM(60, return_sequences=True, input_shape=(7, len(['close']))))
model.add(LSTM(120, return_sequences=False))
model.add(Dense(20))
model.add(Dense(1))

BATCH_SIZE = 8
EPOCHS = 80

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
print(X.shape)
print(Y.shape)
model.fit(X, Y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1)

model.save('./model/NoScore_' + stockTicker + '.keras')

model.summary()

In [ ]:
# GET PREDICTIONS
last_sequence = last_sequence[-N_STEPS:]
last_sequence = np.expand_dims(last_sequence, axis=0)

print(last_sequence)
prediction = model.predict(last_sequence)
prediction